<a href="https://colab.research.google.com/github/AmlanSamanta/TensorFlowExercises/blob/main/TFQuickStartExpert_mnistDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print("Tensorflow version: ", tf.__version__)

from tensorflow.keras.layers import Dense,Flatten,Conv2D
from tensorflow.keras import Model

Tensorflow version:  2.9.2


In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train/255.0, x_test/255.0
x_train, x_test = x_train[...,tf.newaxis].astype("float32"), x_test[...,tf.newaxis].astype("float32")

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32,3,activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self,x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [4]:
model = MyModel()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [5]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [6]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
       predictions = model(images,training=True)
       loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [7]:
  EPOCHS = 5

  for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch+1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result()*100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result()*100} '
    )

Epoch 1, Loss: 0.13263560831546783, Accuracy: 95.97166442871094, Test Loss: 0.05942302569746971, Test Accuracy: 98.04000091552734 
Epoch 2, Loss: 0.04010090231895447, Accuracy: 98.72166442871094, Test Loss: 0.05053010582923889, Test Accuracy: 98.3699951171875 
Epoch 3, Loss: 0.021043898537755013, Accuracy: 99.32167053222656, Test Loss: 0.05899982154369354, Test Accuracy: 98.3499984741211 
Epoch 4, Loss: 0.013806881383061409, Accuracy: 99.52666473388672, Test Loss: 0.06089559197425842, Test Accuracy: 98.19999694824219 
Epoch 5, Loss: 0.008437508717179298, Accuracy: 99.71666717529297, Test Loss: 0.06394696235656738, Test Accuracy: 98.41999816894531 
